In [1]:
from data.data_reading import data_reading
from data.data_processing import data_processing
from ruspy.estimation.estimation_transitions import estimate_transitions
import numpy as np
from scipy.optimize import minimize
from scipy.stats import chi2

In [2]:
beta = 0.9999
init_dict = {
    "groups": "group_4",
    "binsize": 1000,
}

In [3]:
data_reading()
repl_data = data_processing(init_dict)

In [4]:
trans_results = estimate_transitions(repl_data)
print(trans_results)

{'state_count': array([[29, 12, 10, ...,  0,  0,  0],
       [ 0, 12,  2, ...,  0,  0,  0],
       [ 0,  0,  6, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ...,  0,  0,  1],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0]]), 'x': array([2.86579683e-02, 8.10810811e-02, 2.79123952e-01, 2.46039143e-01,
       1.81733458e-01, 1.30242311e-01, 4.65983225e-02, 3.72786580e-03,
       4.65983225e-04, 6.98974837e-04, 9.31966449e-04, 4.65983225e-04,
       2.32991612e-04]), 'fun': 7581.755518761861, 'trans_count': array([ 123,  348, 1198, 1056,  780,  559,  200,   16,    2,    3,    4,
          2,    1])}


In [5]:
def loglike(trans_probs, transition_count):
    trans_probs_scale = trans_probs - np.max(trans_probs)
    log_p  = np.log(np.exp(trans_probs_scale) / np.sum(np.exp(trans_probs_scale)))
    return - np.sum(np.multiply(transition_count, log_p))

In [6]:
scale = 388

In [7]:
res = minimize(loglike, [0.2] * len(trans_results["x"]), args=trans_results["trans_count"] / scale)

In [8]:
params_internal = res["x"]
cov_internal = res["hess_inv"]
p_external = np.exp(res["x"]) / np.sum(np.exp(res["x"]))

In [9]:
trans_results["x"] - p_external

array([ 5.49742141e-08,  1.41581710e-08,  1.48564346e-08,  1.91965391e-09,
        4.27612097e-08, -4.70474770e-08, -4.44653534e-08,  7.84237646e-09,
       -7.32534274e-08,  7.28785590e-08,  1.49562359e-09, -7.93807326e-08,
        3.32607479e-08])

In [10]:
def Kullback_Leibler(p, p_ml):
    """
    This function calculate the Kullback Leibler Divergence of p and p_ml.
    :param p:
    :param p_ml:
    :return:
    """
    return np.sum(np.log(np.divide(p, p_ml)))

In [11]:
obs_per_state = 4292 / scale
rho = chi2.ppf(0.5, len(p_external) - 1) / (2 * obs_per_state)

In [12]:
sum = 0
for i in range(1000):
    draw = np.random.multivariate_normal(params_internal, cov_internal)
    p = np.exp(draw) / np.sum(np.exp(draw))
    if Kullback_Leibler(p, p_external) > rho:
        sum += 1
sum / 1000

0.024